# Домашнее задание по теме «Машинное обучение»

Это домашнее задание состоит из двух частей: теоретической и практической. В первой части нужно изучить методы автоматического отбора признаков и решить 3 задачи.

В практической части нужно решить комплексную задачу регрессии на имеющихся данных.

## Подготовка

In [1]:
# Импортируй библиотеки
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

## Теоретическая часть

> **Важно.** Чтобы пройти теоретическую часть, изучи документации методов, которые описаны ниже, и прочитай любые материалы по теме, которые сможешь найти.

Аналитики данных часто исследуют сферы, функции, методы и подходы, с которыми они никогда не работали. Умение искать информацию, читать документацию и разбираться в теме с нуля — ключевые навыки для работы с данными.

Ранее мы с вами уже разбирали техники, которые используются при исследовании и предобработки данных. Одна из таких техник — отбор признаков.

> **Отбор признаков** — это процесс выбора признаков, которые способны наиболее положительно повлиять на точность решения при построении модели.

Этот этап важен:

* для уменьшения размерности данных;
* улучшения интерпретируемости модели;
* ускорения обучения;
* устойчивости модели.

В машинном обучении существует несколько подходов к отбору признаков. Принято разделять их на фильтрационные, оберточные (методы обертки) и встроенные методы.

### Фильтрационные методы (Filter Methods)

Фильтрационные методы оценивают каждый признак по отдельности на основе их статистических свойств. В результате определяется значимость признака. При этом не учитывается модель обучения, которая будет выбрана позже.

<!-- Фильтрационные методы оценивают каждый признак отдельно, без учета модели, и используют статистические методы для определения значимости признака. -->

**Пример**: корреляция, статистический тест χ². В Python зачастую реализуется через функцию [SelectKBest](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_selection.SelectKBest.html).

In [2]:
# Пример использования SelectKBest для задачи регрессии
# X_selected = SelectKBest(score_func=f_regression, k=5).fit_transform(X, y)

### Методы обёртки (Wrapper Methods)

Методы обёртки смотрят на производительность модели (метрику качества) для оценки качества набора признаков.

**Пример**: рекурсивное исключение признаков ([RFE](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_selection.RFE.html)).

In [3]:
# # Пример использования RFE для задачи регрессии
# model = LinearRegression()
# selector = RFE(model, n_features_to_select=5, step=1)
# X_selected = selector.fit_transform(X, y)

### Встроенные методы (Embedded Methods)

Некоторые модели машинного обучения имеют встроенный функционал для отбора признаков во время обучения модели.

**Пример**: [Lasso](https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.Lasso.html)-регрессия, деревья решений.

In [4]:
# # Пример использования Lasso для задачи регрессии
# lasso = Lasso(alpha=0.01)
# lasso.fit(X, y)
# selected_features = lasso.coef_ != 0

### Задача 1 [1,5 балла]

In [5]:
!gdown --id 1QM0-BH2R0YrwwDjPzRAqNdIOAUO1Mv-R -O ../Datasets/2.2.csv

/home/rostislav-lokhov/Homeworks/AI/.venv/lib/python3.12/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1QM0-BH2R0YrwwDjPzRAqNdIOAUO1Mv-R
To: /home/rostislav-lokhov/Homeworks/AI/Datasets/2.2.csv
100%|███████████████████████████████████████| 49.1k/49.1k [00:00<00:00, 470kB/s]


In [6]:
boston_df = pd.read_csv('../Datasets/2.2.csv', header=None, sep='\s+')

column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
boston_df.columns = column_names
boston_df.head()

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_134324/458718585.py:1: SyntaxWarning: invalid escape sequence '\s'
  boston_df = pd.read_csv('../Datasets/2.2.csv', header=None, sep='\s+')


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [7]:
boston_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


* Используя `SelectKBest` и `score_func = f_regression`, отбери 3 наиболее важных признака в наборе данных `boston_df`. В ответе запиши названия получившихся признаков. **[0,5 балла]**



In [8]:
selector =  SelectKBest(score_func=f_regression, k=3)
X = selector.fit_transform(X=boston_df.drop('MEDV', axis=1), y=boston_df['MEDV'])
mask = selector.get_support()
selected_indices = np.where(mask)[0]
selected_feature_names = boston_df.columns[selected_indices]
selected_feature_names

Index(['RM', 'PTRATIO', 'LSTAT'], dtype='object')

* Используя RFE и модель Linear Regression, отбери 3 наиболее важных признака в наборе данных `boston_df`. В ответе запиши названия получившихся признаков. **[0,5 балла]**

In [9]:
model = LinearRegression()
selector = RFE(model, n_features_to_select=3, step=1)
X = selector.fit_transform(boston_df.drop('MEDV', axis=1), boston_df['MEDV'])
mask = selector.get_support()
selected_indices = np.where(mask)[0]
selected_feature_names = boston_df.columns[selected_indices]
selected_feature_names

Index(['CHAS', 'NOX', 'RM'], dtype='object')

* Используя Lasso c параметрами `alpha = 0.01` и `random_state = 42`, выведи название признака с наибольшим по модулю коэффициентом значимости. **[0,5 балла]**

In [10]:
lasso = Lasso(alpha=0.01, random_state=42)
lasso.fit(boston_df.drop('MEDV', axis=1), boston_df['MEDV'])
selected_features = lasso.coef_ != 0
selected_feature_indices = [i for i, coef in enumerate(selected_features) if coef != 0]
selected_features_names = boston_df.columns[selected_feature_indices]
selected_features_names

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

## Практическая часть

In [11]:
!gdown --id 1pAxawAKgsatcD7dwY2RRKEubUY7RRo-J -O ../Datasets/2.3.csv

/home/rostislav-lokhov/Homeworks/AI/.venv/lib/python3.12/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1pAxawAKgsatcD7dwY2RRKEubUY7RRo-J
To: /home/rostislav-lokhov/Homeworks/AI/Datasets/2.3.csv
100%|██████████████████████████████████████| 3.38M/3.38M [00:01<00:00, 2.34MB/s]


In [12]:
data = pd.read_csv('../Datasets/2.3.csv')
data.head()

,Unnamed: 0,Median_Income,House_Age,Average_Rooms,Average_Bedrooms,Population,Average_Occupancy,Longitude,Median_House_Value,Very_important_feature,Income_Category,Year_Built,Economic_Factor,Log_Latitude
0,0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,-122.23,4.526,0.496714,Middle,1992,0.000000,3.660480
1,1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,-122.22,3.585,-0.138264,Middle,1960,0.000485,3.659965
2,2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,-122.24,3.521,0.647689,Middle,1911,0.000969,3.659708
3,3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,-122.25,3.413,1.523030,Middle,1912,0.001454,3.659708
4,4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,-122.25,3.422,-0.234153,Lower-Middle,1957,0.001938,3.659708


In [13]:
data.shape

(20640, 14)

Тебе уже известен набор данных Boston Housing. В практической части домашнего задания ты поработаешь с его аналогом — набором данных California Housing.

* `Median_Income` — средний доход на домохозяйство в данном районе.
* `Unnamed: 0` — какой-то странный артефакт.
* `House_Age` — возраст дома (разница между годом постройки и текущим годом).
* `Average_Rooms `— cреднее количество комнат в доме.
* `Average_Bedrooms` — cреднее количество спальных комнат в доме.
* `Population` — число жителей в районе, где расположен дом.
* `Average_Occupancy` — среднее количество людей, проживающих в одном доме.
* `Longitude` — географическая долгота данного региона.
* `Median_House_Value` — cредняя стоимость домов в районе (**целевая переменная**).
* `Very_important_feature` — какой-то странный признак.
* `Income_Category` — категория дохода в домохозяйстве (например, "Low", "Middle", "High").
* `Year_Built` — год постройки домов.
* `Economic_Factor` — экономический фактор, отражающий состояние экономики в момент постройки дома.
* `Log_Latitude` — логарифм географической широты. Мы знаем, что логарифмирование проводилось через функцию `np.log1p`.

In [14]:
print(data.describe())
print(data.info())
mumeric_cols = data.select_dtypes(include=['float', 'int'])
categorical_cols = data.select_dtypes(include=['object'])

         Unnamed: 0  Median_Income     House_Age  Average_Rooms  \
count  20640.000000   20640.000000  20640.000000   20640.000000   
mean   10319.500000       3.870671     28.639486       5.429000   
std     5958.399114       1.899822     12.585558       2.474173   
min        0.000000       0.499900      1.000000       0.846154   
25%     5159.750000       2.563400     18.000000       4.440716   
50%    10319.500000       3.534800     29.000000       5.229129   
75%    15479.250000       4.743250     37.000000       6.052381   
max    20639.000000      15.000100     52.000000     141.909091   

       Average_Bedrooms    Population  Average_Occupancy     Longitude  \
count      20640.000000  20640.000000       20640.000000  20640.000000   
mean           1.096675   1425.476744           3.070655   -119.569704   
std            0.473911   1132.462122          10.386050      2.003532   
min            0.333333      3.000000           0.692308   -124.350000   
25%            1.006079   

### Задача 2 [0,25 балла]

Раздели данные на `train`, `val` и `test` подвыборки в соотношении 70/15/15. `random state` должен быть равен 42 там, где это необходимо. В ответе запиши индекс первой строки в подвыборке `X_test`. **[0,25 балла]**



In [15]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    data, data['Median_House_Value'], random_state=42, train_size=0.7)
X_val, X_test, y_val, y_test = train_test_split(
    X_val, y_val, random_state=42, train_size=0.5)

### Задача 3 [0,25 балла]

Построй наивную модель (допустим, мы всегда предсказываем среднее значение целевой переменной). В ответе запиши значение MAE, получаемое при сравнении подобных предсказаний с `y_test` c точностью до 4 знаков после запятой. **[0,25 балла]**


In [16]:
from sklearn.metrics import mean_absolute_error
print(round(mean_absolute_error(y_test, np.full(y_test.shape, y_train.mean())), 4))

0.921


### Задача 4 [8 баллов]

Тебя ждёт первое большое практическое задание. Вспомни всё, что мы проходили на курсе.

> **Важно.** Обязательно используй код из предыдущих домашних заданий и семинаров — там много полезного.

Выше тебе нужно было загрузить данные California Housing, разбить их на подвыборки, построить наивную модель. Твоя задача — на основе этих данных решить задачу регрессии, максимально улучшив метрику MAE.

Можешь пользоваться всем набором изученных инструментов и техник: поиском инсайдов через EDA, различными black-box моделями, Feature Engineering, обработкой признаков, отбором признаков и другими.

### Рекомендации

* Подробно комментируй код, ход работы и мыслей.
* Погрузись в данные, попробуй понять «физический смысл» признаков.
* Следуй стандартной последовательности действий при работе над ML-решением.
* Получи 0,1-0,2 MAE через LinearRegression, прежде чем работать с другими моделями.
* Следи за переобучением. Там, где это уместно, распиши, почему ты считаешь, что модель переобучилась/не переобучилась.
* Используй `random_state` там, где это уместно.

### Система оценивания

* Проведен EDA анализ, расписаны выводы о ключевых особенностях данных (пропуски, выбросы, категориальные признаки, корреляции). **[2 балла]**
* Продемонстрированы навыки визуализации (построена хотя бы 1 гистограмма, plotly geomap, тепловая матрица корреляций). **[1 балл]**
* 0,1-0,2 MAE получено только через LinearRegression и работу с данными. **[2 балла]**
* 0,06-0,099 MAE получено без переобучения. **[1 балл]**;
* Продемонстрированы навыки работы с автоматизированным поиском гиперпараметров. **[1 балл]**
* Продемонстрированы навыки работы с Feature Engineering (код и обоснование того или иного действия). **[1 балл]**
* Итоговый лучший MAE score меньше 0,06 (и модель выдает близкие значения метрики на валидации и тесте). **[2 дополнительных балла]**

> **Важно**. Если в коде есть ошибки, которые привели к успешному решению задания (например, утечка данных в любом виде), то проверяющий имеет право не засчитывать баллы за те пункты, на которые эти ошибки напрямую повлияли.

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import optuna

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


def onehot_income_category(x: str) -> int:
    dct = {'Low': -2, 'Lower-Middle': -1, 'Middle': 0, 'Upper-Middle': 1, 'High': 2}
    return dct[x]

df = pd.read_csv('../Datasets/2.3.csv')
df['Diagonal'] = (df['Longitude'] + np.expm1(df['Log_Latitude']))
df['Bed_Per_Room'] = df['Average_Bedrooms'] / df['Average_Rooms']
df.drop("Unnamed: 0", axis=1, inplace=True)
df.drop("Year_Built", axis=1, inplace=True)
df.drop("Very_important_feature", axis=1, inplace=True)
df['Income_Category'] = df['Income_Category'].map(onehot_income_category)
df['Income_per_Room'] = df['Median_Income']/df['Average_Rooms']
df['Income_per_Bedroom'] = df['Median_Income']/df['Average_Bedrooms']
df['Rooms_per_Person'] = df['Average_Rooms']/df['Average_Occupancy']
df['Bedroms_per_Room'] = df['Average_Bedrooms']/df['Average_Rooms']
df['Population_Density'] = df['Longitude']-df['Log_Latitude']
df['Income_Age'] = df['House_Age']*df['Median_Income']
print(df.info(), df.describe(include="all"))

print(df.corrwith(df['Median_House_Value']).sort_values())

X = df.drop('Median_House_Value', axis=1)
y = df['Median_House_Value']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, train_size=0.8, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, train_size=0.5, random_state=42)


numeric_cols = [col for col in X_train.select_dtypes(include=['float', 'int']).columns
                if col != 'Income_Category']

scaler = MinMaxScaler()
scaler_target = MinMaxScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_val[numeric_cols] = scaler.transform(X_val[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

y_train = scaler_target.fit_transform(
    y_train.values.reshape(-1, 1))
y_val = scaler_target.transform(
    y_val.values.reshape(-1, 1))
y_test = scaler_target.transform(
    y_test.values.reshape(-1, 1))


def show_metrics(train_pred, y_train, y_val, val_pred, y_test, test_pred):
    print(f"RMSE_train: {root_mean_squared_error(y_train, train_pred)}")
    print(f"RMSE_val: {root_mean_squared_error(y_val, val_pred)}")
    print(f"RMSE_test: {root_mean_squared_error(y_test, test_pred)}")
    print()
    print(f"MAE_train: {mean_absolute_error(y_train, train_pred)}")
    print(f"MAE_val: {mean_absolute_error(y_val, val_pred)}")
    print(f"MAE_test: {mean_absolute_error(y_test, test_pred)}")
    print(f"R^2 train: {r2_score(y_train, train_pred)}")
    print(f"R^2 val: {r2_score(y_val, val_pred)}")
    print(f"R^2 test: {r2_score(y_test, test_pred)}")



def fit_predict(model, X_train, y_train, X_val, y_val, X_test, y_test):
    model.fit(X_train, y_train.ravel()) 
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    test_pred = model.predict(X_test)
    print(f"MODEL: {model}")
    show_metrics(train_pred, y_train, y_val, val_pred, y_test, test_pred)
    return model


fit_predict(LinearRegression(n_jobs=-1), X_train,
            y_train, X_val, y_val, X_test, y_test)

fit_predict(RandomForestRegressor(n_jobs=-1), X_train,
            y_train, X_val, y_val, X_test, y_test)

def objective(trial):
    rf_params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 200),
        'max_depth': trial.suggest_int('max_depth', 5, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
        'random_state': 42,
        'n_jobs': -1
    }
    model = RandomForestRegressor(**rf_params)
    model.fit(X_train, y_train.ravel()) 
    val_pred = model.predict(X_val)
    val_rmse = mean_absolute_error(y_val, val_pred)
    return val_rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print(f"Best hyperparameters: {best_params}")

best_rf_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
fit_predict(best_rf_model, X_train, y_train, X_val, y_val, X_test, y_test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Median_Income       20640 non-null  float64
 1   House_Age           20640 non-null  float64
 2   Average_Rooms       20640 non-null  float64
 3   Average_Bedrooms    20640 non-null  float64
 4   Population          20640 non-null  float64
 5   Average_Occupancy   20640 non-null  float64
 6   Longitude           20640 non-null  float64
 7   Median_House_Value  20640 non-null  float64
 8   Income_Category     20640 non-null  int64  
 9   Economic_Factor     20640 non-null  float64
 10  Log_Latitude        20640 non-null  float64
 11  Diagonal            20640 non-null  float64
 12  Bed_Per_Room        20640 non-null  float64
 13  Income_per_Room     20640 non-null  float64
 14  Income_per_Bedroom  20640 non-null  float64
 15  Rooms_per_Person    20640 non-null  float64
 16  Bedr

[I 2025-02-12 18:26:16,054] A new study created in memory with name: no-name-2555a37c-bfd3-4df5-bfb9-5a3e527e8ba0


MODEL: RandomForestRegressor(n_jobs=-1)
RMSE_train: 0.036511764889254276
RMSE_val: 0.09548368774368743
RMSE_test: 0.09650831629533929

MAE_train: 0.023219935708718324
MAE_val: 0.06136572421910255
MAE_test: 0.06318810060489162
R^2 train: 0.9765418719281376
R^2 val: 0.8371315640781378
R^2 test: 0.8319070383599595


[I 2025-02-12 18:26:29,974] Trial 0 finished with value: 0.06238920039352617 and parameters: {'n_estimators': 181, 'max_depth': 16, 'min_samples_split': 9, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.06238920039352617.
[I 2025-02-12 18:26:43,906] Trial 1 finished with value: 0.061910600040872954 and parameters: {'n_estimators': 175, 'max_depth': 17, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.061910600040872954.
[I 2025-02-12 18:26:50,671] Trial 2 finished with value: 0.06341219640737225 and parameters: {'n_estimators': 102, 'max_depth': 14, 'min_samples_split': 2, 'min_samples_leaf': 1}. Best is trial 1 with value: 0.061910600040872954.
[I 2025-02-12 18:27:02,341] Trial 3 finished with value: 0.06222179845594632 and parameters: {'n_estimators': 175, 'max_depth': 17, 'min_samples_split': 9, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.061910600040872954.
[I 2025-02-12 18:27:12,168] Trial 4 finished with value: 0.062129406343524415 an

Best hyperparameters: {'n_estimators': 165, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 1}
MODEL: RandomForestRegressor(max_depth=20, min_samples_split=4, n_estimators=165,
                      n_jobs=-1, random_state=42)
RMSE_train: 0.040565991896593816
RMSE_val: 0.09538008398399303
RMSE_test: 0.0959727063281077

MAE_train: 0.026087151464046533
MAE_val: 0.06133193694641007
MAE_test: 0.06273392256583081
R^2 train: 0.9710431111048465
R^2 val: 0.8374848103408012
R^2 test: 0.833767653792988


RandomForestRegressor(max_depth=20, min_samples_split=4, n_estimators=165,
                      n_jobs=-1, random_state=42)